In [ ]:
# Google Colabでライブラリをアップロードする
from google.colab import files
uploaded = files.upload()

In [ ]:
# Google Colabでドライブのデータを使う
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Google Colabでライブラリをインストールする
!pip install japanize_matplotlib bottleneck tslearn

In [523]:
# 自作関数
import MasterResearchFunction as mr

# 基本ライブラリ
import csv
import math
import os
import pickle
import random
import re
import statistics
from datetime import datetime, timedelta
from decimal import Decimal

# 数値計算とデータ処理
import bottleneck as bn
import numpy as np
import pandas as pd

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras.layers import (
    Activation,
    Add,
    Conv1D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling1D,
    Input,
    LayerNormalization,
    LSTM,
    Masking,
    MaxPooling1D,
    MultiHeadAttention,
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


# プロットと可視化
import japanize_matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

# その他のライブラリ
from fastdtw import fastdtw
from scipy import signal, stats
from scipy.interpolate import interp1d, UnivariateSpline
from scipy.signal import savgol_filter
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import fcluster, linkage
from tslearn.metrics import dtw_path

In [524]:
# 定数の定義
NAMES = ['sakamoto', 'watabe', 'kotera', 'nakazawa', 'mizuno', 'todaka', 'takagawa', 'uchida']
LABELS = ['circle', 'cross', 'tri', 'check']
COLUMNS_TO_DROP = [
    'Sensor', 'Participant name', 'Event', 'Event value',
    'Eye movement type', 'Eye movement type index', 'Ungrouped',
    'Validity left', 'Validity right', 'Gaze event duration', 'Gaze2D_Distance',
    'Fixation_Distance', 'Gaze3D_Distance', 'Pupil_Diameter_Change',
    'GazeDirection_Distance', 'PupilPosition_Distance'
]
BASE_DIR = "/Users/hinase/CodeChord/MasterResearch/datasets/new"
Hz = 100
#各被験者がジェスチャを行った順番(NAMES順)
ROUTINE_LABELS = [
    [1, 2, 3, 4],
    [2, 4, 3, 1],
    [4, 3, 1, 2],
    [2, 4, 3, 1],
    [1, 4, 3, 2],
    [3, 2, 1, 4],
    [3, 1, 2, 4],
    [2, 3, 1, 4]
]
# 数字からラベルへの変換マッピング
label_map = {1: 'circle', 2: 'cross', 3: 'tri', 4: 'check'}
# ラベル辞書を作成
routine_label_dict = {name: [label_map[num] for num in labels] for name, labels in zip(NAMES, ROUTINE_LABELS)}

In [ ]:
#ジェスチャの教師データの読み込み
trimmed_data_dict = mr.load_trimmed_gesture_data(NAMES, BASE_DIR)
#教師データの最小の長さと最大の長さを格納
length_extremes_dict = mr.calculate_length_extremes(trimmed_data_dict)

In [ ]:
#実験ルーティンのデータの読み込みと，各ジェスチャの実行部分のインデックスの抽出
routine_data_dict = {}
grouped_intervals = {}
group_label_mapping = {}

for name in NAMES:
    print(f"Processing {name}...")

    # データの読み込みと前処理
    routine_data_dict[name] = mr.process_apple_watch_csv(f'{BASE_DIR}/{name}/motion/{name}_new_routine.csv')
    a = mr.find_true_intervals(routine_data_dict[name])

    # インターバルの開始インデックスを取得
    start_indices = np.array([interval[0] for interval in a]).reshape(-1, 1)

    # 階層的クラスタリング
    Z = linkage(start_indices, method='ward')
    cluster_labels = fcluster(Z, t=4, criterion='maxclust')  # クラスタ数を4に固定

    # クラスタごとにインターバルをグループ化
    interval_groups = {i: [] for i in range(1, 5)}
    for interval, label in zip(a, cluster_labels):
        interval_groups[label].append(interval)

    # グループの順序をインターバルの開始インデックスに基づいてソート
    sorted_groups = sorted(interval_groups.items(), key=lambda x: min(interval[0] for interval in x[1]))
    sorted_interval_groups = {i + 1: group for i, (_, group) in enumerate(sorted_groups)}

    # グループを保存
    grouped_intervals[name] = sorted_interval_groups

    # グループとラベルを関連付け
    group_label_mapping[name] = {label: sorted_interval_groups[group_id]
                                for group_id, label in enumerate(routine_label_dict[name], start=1)}

    # 結果を出力
    print(f"\n{name} のグループとラベル対応:")
    for label, intervals in group_label_mapping[name].items():
        print(f"  {label}: {intervals}")

# 'nakazawa' の 'check' に対応する最初のインデックスの組を削除
if 'nakazawa' in group_label_mapping and 'check' in group_label_mapping['nakazawa']:
    if len(group_label_mapping['nakazawa']['check']) > 0:
        removed_interval = group_label_mapping['nakazawa']['check'].pop(0)
        print(f"'nakazawa' の 'check' から削除されたインターバル: {removed_interval}")
    else:
        print("'nakazawa' の 'check' に削除するインターバルがありません。")
else:
    print("'nakazawa' または 'check' に対応するデータが見つかりません。")

In [ ]:
#ジェスチャ実行時の-1秒から+1秒の範囲を抜き出す
extracted_data_dict = {}

for name in NAMES:
    print(f"Processing {name}...")
    extracted_data_dict[name] = {}

    for label in LABELS:
        print(f"Processing label: {label}...")

        # 該当するインターバルを取得
        intervals = group_label_mapping[name][label]

        # 最初のインデックスと最後のインデックスを計算
        start_idx = intervals[0][0] - 500
        end_idx = intervals[-1][1] + 500

        # インデックスの範囲を調整（負のインデックス防止）
        start_idx = max(0, start_idx)
        end_idx = min(len(routine_data_dict[name]), end_idx)

        # データを抜き出し
        extracted_data = routine_data_dict[name].iloc[start_idx:end_idx].reset_index(drop=True)

        # 抜き出したデータを辞書に格納
        extracted_data_dict[name][label] = extracted_data

        print(f"Extracted data for {name} - {label}: start={start_idx}, end={end_idx}, length={len(extracted_data)}")

In [ ]:
# プロットの関数
def plot_extracted_data(extracted_data_dict, name, label):
    """
    抽出データをプロットし、Marking が True の部分をハイライト。

    Parameters:
    - extracted_data_dict (dict): 抽出されたデータの辞書。
    - name (str): 被験者の名前。
    - label (str): ジェスチャーのラベル。
    """
    data = extracted_data_dict[name][label]

    if data.empty:
        print(f"{name} の {label} にデータがありません。")
        return

    # 各特徴量をプロット
    plt.figure(figsize=(12, 6))

    plt.plot(data['AccelerationX'], label='AccelerationX', color='red', alpha=0.7)
    plt.plot(data['AccelerationY'], label='AccelerationY', color='green', alpha=0.7)
    plt.plot(data['AccelerationZ'], label='AccelerationZ', color='blue', alpha=0.7)
    plt.plot(data['EuclideanNorm'], label='EuclideanNorm', color='purple', linewidth=1.5)

    # Marking が True の箇所をハイライト
    marking = data['Marking']
    for i in range(len(marking)):
        if marking.iloc[i]:
            plt.axvspan(i, i + 1, color='yellow', alpha=0.3)

    # ラベルとタイトル
    plt.title(f"{name} - {label}", fontsize=16)
    plt.xlabel("Time Step", fontsize=12)
    plt.ylabel("Value", fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)

    # プロット表示
    plt.tight_layout()
    plt.show()

# プロット実行
for name in NAMES:
    for label in LABELS:
        print(f"Plotting {name} - {label}...")
        plot_extracted_data(extracted_data_dict, name, label)

In [ ]:
LABELS[jesutya]
cross27.5
check4.3

In [ ]:
NAMES[namae]

In [ ]:
LABELS[jesutya]

In [ ]:
namae = 0
jesutya = 2
Th = 9.1
Ths = [Th, Th, Th]
b = mr.three_axis_spring(extracted_data_dict[NAMES[namae]][LABELS[jesutya]], trimmed_data_dict[NAMES[namae]][LABELS[jesutya]], Ths, 'acc')

In [ ]:
b

In [1010]:
c = mr.filter_and_combine_segments(b, Hz, min_time=length_extremes_dict[NAMES[namae]][LABELS[jesutya]]['min_length']*0.5/Hz, max_time=length_extremes_dict[NAMES[namae]][LABELS[jesutya]]['max_length']*2.5/Hz)

In [ ]:
c

In [ ]:
# グラフプロット関数
def plot_detected_segments_with_highlights(data, marking_intervals, detected_intervals):
    """
    データの区間をハイライトしてプロットする関数。

    Parameters:
    - data (pd.DataFrame): プロットするデータ。
    - marking_intervals (list of tuple): ['Marking']がTrueであるインデックスのリスト。
    - detected_intervals (list of tuple): SPRINGで検出されたインデックスのリスト。
    """
    plt.figure(figsize=(12, 6))

    # 元データをプロット
    plt.plot(data.index, data['EuclideanNorm'], label='EuclideanNorm', color='blue')

    # ['Marking']がTrueの区間を黄色でハイライト
    for start, end in marking_intervals:
        plt.axvspan(start, end, color='yellow', alpha=0.3, label='Marking=True' if start == marking_intervals[0][0] else "")

    # SPRINGで検出された区間を青色でハイライト
    for start, end in detected_intervals:
        plt.axvspan(start, end, color='cyan', alpha=0.4, label='SPRING Detected' if start == detected_intervals[0][0] else "")

    # グラフ設定
    plt.title("Detected Segments with Highlights", fontsize=16)
    plt.xlabel("Time Step", fontsize=12)
    plt.ylabel("EuclideanNorm", fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    plt.tight_layout()
    plt.show()

# マーキング区間の抽出
marking_intervals = []
current_start = None
for i in range(len(extracted_data_dict[NAMES[namae]][LABELS[jesutya]]['Marking'])):
    if extracted_data_dict[NAMES[namae]][LABELS[jesutya]]['Marking'].iloc[i] and current_start is None:
        current_start = i
    elif not extracted_data_dict[NAMES[namae]][LABELS[jesutya]]['Marking'].iloc[i] and current_start is not None:
        marking_intervals.append((current_start, i - 1))
        current_start = None
if current_start is not None:
    marking_intervals.append((current_start, len(extracted_data_dict[NAMES[namae]][LABELS[jesutya]]['Marking']) - 1))

# 検出された区間
detected_intervals = c

# グラフプロット
plot_detected_segments_with_highlights(
    extracted_data_dict[NAMES[namae]][LABELS[jesutya]], marking_intervals, detected_intervals
)

In [ ]:
# データとラベルを格納するリストの初期化
X_filled = []
y_labels = []
true_labels = []
# データとラベルを格納する辞書の初期化
data_dict = {}
label_dict = {}
true_label_dict = {}

for name in NAMES:
    print(f"Processing {name}...")

    # データの読み込みと前処理
    eye_data, gesture_labels = mr.load_gesture_eye_data_pickle(name, LABELS, COLUMNS_TO_DROP)
    data_dict[name] = {'gesture': eye_data, 'non_gesture': []}
    label_dict[name] = {'gesture': gesture_labels, 'non_gesture': []}
    true_label_dict[name] = {'gesture': ['gesture'] * len(eye_data), 'non_gesture': []}

    # モーションデータとアイデータの読み込み
    motion_routine_data, eye_routine_data = mr.load_motion_and_eye_data(name)
    eye_non_gesture_data = mr.process_tobii_csv(f'datasets/new/{name}/eye/{name}_non_gesture.csv')

    # gestureデータの長さを算出
    if isinstance(eye_data, list):
        lengths = [df.shape[0] for df in eye_data]  # 各データフレームの行数を取得
        longest_length = max(lengths)
        shortest_length = min(lengths)
    else:
        longest_length = eye_data.shape[0]
        shortest_length = eye_data.shape[0]

    print(f"{name} の gesture データの長さ: 最長 {longest_length}, 最短 {shortest_length}")

    # ランダムな長さでデータを抽出
    NON_GESTURE = 180
    random_non_gesture_data = []

    # ランダムな長さでデータを抽出（重複を防ぐために利用可能なインデックスを管理）
    available_indices = list(range(eye_non_gesture_data.shape[0]))
    random_non_gesture_data = []

    for _ in range(NON_GESTURE):
        # ランダムな長さを決定
        random_length = random.randint(shortest_length, longest_length)

        # 利用可能なインデックスからランダムに開始位置を選択
        possible_start_indices = [
            idx for idx in available_indices if idx + random_length <= len(available_indices)
        ]
        if not possible_start_indices:
            print(f"{name} の non-gesture データに十分なインデックスがありません。")
            break

        start_idx = random.choice(possible_start_indices)
        end_idx = start_idx + random_length

        # 抽出したデータを追加
        random_non_gesture_data.append(eye_non_gesture_data.iloc[start_idx:end_idx])

        # 使用済みインデックスを削除
        available_indices = [idx for idx in available_indices if idx < start_idx or idx >= end_idx]

    # 抽出したデータを辞書に格納
    data_dict[name]['non_gesture'] = random_non_gesture_data
    label_dict[name]['non_gesture'] = ['non_gesture'] * len(random_non_gesture_data)
    true_label_dict[name]['non_gesture'] = ['non_gesture'] * len(random_non_gesture_data)

    print(f"{name} の non-gesture データから {len(random_non_gesture_data)} 個のランダムデータを抽出しました。")

    # ジェスチャーデータの統合
    for seq, lbl, true_lbl in zip(eye_data, gesture_labels, true_label_dict[name]['gesture']):
        if seq is not None and len(seq) > 0:
            X_filled.append(seq)
            y_labels.append(lbl)
            true_labels.append(true_lbl)
        else:
            print(f"{name} のジェスチャーシーケンスが空です。対応するラベルをスキップします。")

    # 非ジェスチャーデータの統合
    for seq, lbl, true_lbl in zip(data_dict[name]['non_gesture'], label_dict[name]['non_gesture'], true_label_dict[name]['non_gesture']):
        if isinstance(seq, pd.DataFrame):
            if not seq.empty:
                X_filled.append(seq)
                y_labels.append(lbl)
                true_labels.append(true_lbl)
            else:
                print(f"{name} の非ジェスチャーシーケンスが空です。対応するラベルをスキップします。")
        else:
            print(f"{name} の非ジェスチャーシーケンスが DataFrame ではありません。スキップします。")


In [ ]:
print(f"総データ数: {len(X_filled)}")
print(f"総ラベル数（y_labels）: {len(y_labels)}")
print(f"総真偽ラベル数（true_labels）: {len(true_labels)}")

In [ ]:
# データとラベルを格納するリストの初期化
X_filled = []
y_labels = []
true_labels = []
# データとラベルを格納する辞書の初期化
data_dict = {}
label_dict = {}
true_label_dict = {}

for name in NAMES:
    print(f"Processing {name}...")

    # データの読み込みと前処理
    eye_data, gesture_labels = mr.load_gesture_eye_data_pickle(name, LABELS, COLUMNS_TO_DROP)
    # 変数に保存
    data_dict[name] = {
        'gesture': eye_data,
        'non_gesture': []
    }
    label_dict[name] = {
        'gesture': gesture_labels,
        'non_gesture': []
    }
    true_label_dict[name] = {
        'gesture': ['gesture'] * len(eye_data),
        'non_gesture': []
    }

    # モーションデータとアイデータの読み込みと前処理
    motion_routine_data, eye_routine_data = mr.load_motion_and_eye_data(name)
    eye_non_gesture_data = mr.process_tobii_csv(f'datasets/new/{name}/eye/{name}_non_gesture.csv')
    # # 変数に保存
    # motion_data = motion_routine_data

    # # 除外インターバルの取得
    # exclusion_intervals = mr.get_exclusion_intervals(motion_data)

    # # データ全体の時間範囲の取得
    # start_time = motion_routine_data['Timestamp'].min()
    # end_time = motion_routine_data['Timestamp'].max()

    # # 利用可能なインターバルの取得
    # available_intervals = mr.get_available_intervals(exclusion_intervals, start_time, end_time)

    # print(f"{name} の利用可能なインターバル数: {len(available_intervals)}")

    # # ランダムなインターバルの抽出（eye_routine_data を使用）
    # extracted_data = mr.extract_random_intervals(
    #     eye_routine_data, available_intervals, MIN_INTERVAL, MAX_INTERVAL, NUM_INTERVALS
    # )

    # # 抽出されたデータを辞書に保存
    # data_dict[name]['non_gesture'] = extracted_data
    # label_dict[name]['non_gesture'] = ['non_gesture'] * len(extracted_data)
    # true_label_dict[name]['non_gesture'] = ['non_gesture'] * len(extracted_data)

    # # ジェスチャーデータの統合
    # for seq, lbl, true_lbl in zip(eye_data, gesture_labels, true_label_dict[name]['gesture']):
    #     if seq is not None and len(seq) > 0:
    #         X_filled.append(seq)
    #         y_labels.append(lbl)
    #         true_labels.append(true_lbl)
    #     else:
    #         print(f"{name} のジェスチャーシーケンスが空です。対応するラベルをスキップします。")

    # # 非ジェスチャーデータの統合
    # for seq, lbl, true_lbl in zip(data_dict[name]['non_gesture'], label_dict[name]['non_gesture'], true_label_dict[name]['non_gesture']):
    #     if isinstance(seq, pd.DataFrame):
    #         if not seq.empty:
    #             X_filled.append(seq)
    #             y_labels.append(lbl)
    #             true_labels.append(true_lbl)
    #         else:
    #             print(f"{name} の非ジェスチャーシーケンスが空です。対応するラベルをスキップします。")
    #     else:
    #         print(f"{name} の非ジェスチャーシーケンスが DataFrame ではありません。スキップします。")

print(f"総データ数: {len(X_filled)}")
print(f"総ラベル数（y_labels）: {len(y_labels)}")
print(f"総真偽ラベル数（true_labels）: {len(true_labels)}")

In [ ]:
# タイムスタンプのギャップを確認
for name in NAMES:
    non_gesture_intervals = data_dict[name]['non_gesture']
    if non_gesture_intervals:
        for idx, eye_routine_data in enumerate(non_gesture_intervals):
            if isinstance(eye_routine_data, pd.DataFrame):
                if not eye_routine_data.empty:
                    time_diffs = eye_routine_data['Timestamp'].diff().dt.total_seconds()
                    large_gaps = time_diffs[time_diffs > 0.05]
                    print(f"{name} の non_gesture_intervals[{idx}] の大きなタイムスタンプギャップ数: {len(large_gaps)}")
                else:
                    print(f"{name} の non_gesture_intervals[{idx}] が空です。")
            else:
                print(f"{name} の non_gesture_intervals[{idx}] が DataFrame ではありません。")

In [ ]:
# 2. 前処理とフィルタリングの適用
filtered_X, filtered_y, filtered_true = mr.preprocess_and_filter_sequences(
    X_filled,
    y_labels,
    true_labels,
    sampling_rate=50.0,
    max_gap_seconds=0.1,
    max_nan_seconds=1.0
)

In [ ]:
# 最終的なデータの確認
print(f"処理後のデータ数: {len(filtered_X)}")
print(f"ラベル配列の長さ（y_labels_scaled）: {len(filtered_y)}")
print(f"真偽ラベル配列の長さ（true_labels_scaled）: {len(filtered_true)}")

In [19]:
X_scaled = []
scaler = StandardScaler()
for i in range(len(filtered_X)):
    X_scaled.append(scaler.fit_transform(filtered_X[i]))

In [20]:
# 4. ラベルのエンコーディング（2値分類用に修正）
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(filtered_true)  # 0または1にエンコード

# **ラベルのワンホットエンコーディングを削除**
# 2値分類では必要ありません
# y_categorical = to_categorical(y_encoded, num_classes=num_classes)

# 5. データの分割（パディングの前に行う）
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.3, random_state=42
)

In [ ]:
for i, seq in enumerate(X_scaled):
    print(f"Sequence {i}: shape {seq.shape}")

In [ ]:
# 6. 訓練データとテストデータでそれぞれ最大シーケンス長を計算
max_length_train = max(len(seq) for seq in X_train_raw)
max_length_test = max(len(seq) for seq in X_test_raw)
max_length = max(max_length_train, max_length_test)

# 7. シーケンスのパディング
X_train_padded = pad_sequences(X_train_raw, maxlen=max_length, padding='post', value=0.0, dtype='float32')
X_test_padded = pad_sequences(X_test_raw, maxlen=max_length, padding='post', value=0.0, dtype='float32')

# 8. パディング後のデータに NaN が含まれていないか確認
print('NaN in X_train_padded:', np.isnan(X_train_padded).any())
print('NaN in X_test_padded:', np.isnan(X_test_padded).any())

# 必要に応じて NaN を 0 で置換
if np.isnan(X_train_padded).any():
    X_train_padded = np.nan_to_num(X_train_padded, nan=0.0)
if np.isnan(X_test_padded).any():
    X_test_padded = np.nan_to_num(X_test_padded, nan=0.0)

In [23]:
def get_positional_encoding(max_len, d_model):
    """
    固定のサイン・コサイン位置エンコーディングを生成する関数。

    Parameters:
    - max_len (int): シーケンスの最大長
    - d_model (int): 埋め込み次元数

    Returns:
    - pos_encoding (np.array): 位置エンコーディング行列
    """
    angle_rads = np.arange(max_len)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model)[np.newaxis, :]//2)) / np.float32(d_model))

    # 偶数次元はsin、奇数次元はcos
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]  # (1, max_len, d_model)
    return tf.cast(pos_encoding, dtype=tf.float32)

from tensorflow.keras.layers import Masking

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    """
    Transformerエンコーダーブロックを構築する関数。
    """
    # 自己注意機構（マスクは自動的に適用される）
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = Dropout(dropout)(x)
    x = LayerNormalization(epsilon=1e-6)(inputs + x)

    # フィードフォワードネットワーク
    x_ff = Dense(ff_dim, activation='relu')(x)
    x_ff = Dense(inputs.shape[-1])(x_ff)
    x_ff = Dropout(dropout)(x_ff)
    out = LayerNormalization(epsilon=1e-6)(x + x_ff)
    return out

def build_transformer_model(max_length, num_features, head_size, num_heads, ff_dim, num_transformer_blocks, dropout):
    """
    Transformerベースの2値分類モデルを構築する関数。
    """
    inputs = Input(shape=(max_length, num_features))

    # マスキングレイヤーの追加（パディング値が0の場合）
    x = Masking(mask_value=0.0)(inputs)

    # 位置エンコーディングの追加
    pos_encoding = get_positional_encoding(max_length, num_features)
    x = x + pos_encoding[:,:max_length, :]

    # Transformerエンコーダーブロックの追加
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    # グローバルプーリング
    x = GlobalAveragePooling1D()(x)
    x = Dropout(dropout)(x)

    # 出力層
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [24]:
from tensorflow.keras.optimizers import Adam

In [ ]:
# モデルの構築
model = build_transformer_model(
    max_length=max_length,
    num_features=X_train_padded.shape[2],
    head_size=64,
    num_heads=8,
    ff_dim=128,
    num_transformer_blocks=1,
    dropout=0.1
)

# モデルのサマリー表示
model.summary()

# モデルのコンパイル
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# モデルの訓練
history = model.fit(
    X_train_padded, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_padded, y_test),
)

In [ ]:
# 4. モデルの評価
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# 5. 予測と結果の表示
y_pred_prob = model.predict(X_test_padded)
y_pred_classes = (y_pred_prob > 0.5).astype(int).flatten()  # 閾値0.5でクラスを決定
y_true_classes = y_test

# ラベルを逆変換（元のクラス名に戻す場合）
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_true_labels = label_encoder.inverse_transform(y_true_classes)

# 分類レポートの表示
print(classification_report(y_true_labels, y_pred_labels))

In [28]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns

In [ ]:
# 混同行列の作成と可視化
conf_mat = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# 1. 特徴量数を取得
num_features = X_train_padded.shape[2]
print(f"特徴量数: {num_features}")

# 2. モデルの構築（2値分類用に修正）
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(max_length, num_features)))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))  # 出力層を1ユニットのシグモイドに変更

# 3. モデルのコンパイルと学習
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train_padded, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_padded, y_test)
)

# 4. モデルの評価
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# 5. 予測と結果の表示
y_pred_prob = model.predict(X_test_padded)
y_pred_classes = (y_pred_prob > 0.5).astype(int).flatten()  # 閾値0.5でクラスを決定
y_true_classes = y_test

# ラベルを逆変換（元のクラス名に戻す場合）
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_true_labels = label_encoder.inverse_transform(y_true_classes)

# 分類レポートの表示
print(classification_report(y_true_labels, y_pred_labels))

In [ ]:
# 特徴量数を取得
num_features = X_train_padded.shape[2]
num_features

In [ ]:
# 4. モデルの構築
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(max_length, num_features)))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='sigmoid'))

# 5. モデルのコンパイルと学習
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train_padded, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_padded, y_test)
)

# 6. モデルの評価
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# 7. 予測と結果の表示
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_true_labels = label_encoder.inverse_transform(y_true_classes)
print(classification_report(y_true_labels, y_pred_labels))

In [ ]:
input_shape = (max_length, num_features)
inputs = Input(shape=input_shape)
x = Masking(mask_value=0.)(inputs)

# num_featuresを定義（入力データの特徴量の次元数）
num_features = input_shape[1]

# Transformerブロックの定義
def transformer_block(x, num_heads, key_dim, ff_dim, rate=0.1):
    # マルチヘッド注意機構
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x, x)
    attn_output = Dropout(rate)(attn_output)

    # 出力次元をnum_featuresに変換
    attn_output = Dense(num_features)(attn_output)

    out1 = Add()([x, attn_output])
    out1 = LayerNormalization(epsilon=1e-6)(out1)

    # フィードフォワードネットワーク
    ffn_output = Dense(ff_dim, activation='relu')(out1)

    # 出力次元をnum_featuresに変換
    ffn_output = Dense(num_features)(ffn_output)

    ffn_output = Dropout(rate)(ffn_output)
    out2 = Add()([out1, ffn_output])
    out2 = LayerNormalization(epsilon=1e-6)(out2)
    return out2

# Transformerブロックの適用
x = transformer_block(x, num_heads=4, key_dim=64, ff_dim=128)

# プーリングと出力層
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
outputs = Dense(num_classes, activation='sigmoid')(x)

# モデルの作成
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
# 5. モデルのコンパイルと学習
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train_padded, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_padded, y_test)
)


In [ ]:
# 6. モデルの評価
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# 7. 予測と結果の表示
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_true_labels = label_encoder.inverse_transform(y_true_classes)
print(classification_report(y_true_labels, y_pred_labels))

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# 8. 特徴量の重要度評価（Permutation Feature Importance）

# 8.1 ベースラインの性能を計算
# テストデータでの予測
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# ベースラインのAccuracyを計算
baseline_accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f'Baseline Accuracy: {baseline_accuracy:.4f}')

# 8.2 特徴量ごとの重要度を計算
num_features = X_test_padded.shape[2]
feature_importances = []

for feature_idx in range(num_features):
    # テストデータをコピー
    X_test_permuted = X_test_padded.copy()

    # 特徴量をランダムにシャッフル（サンプル間でシャッフル）
    permuted_feature = X_test_permuted[:, :, feature_idx].reshape(X_test_permuted.shape[0], -1)
    np.random.shuffle(permuted_feature)
    X_test_permuted[:, :, feature_idx] = permuted_feature.reshape(X_test_permuted.shape[0], X_test_permuted.shape[1])

    # シャッフル後のデータで予測
    y_pred_permuted = model.predict(X_test_permuted)
    y_pred_classes_permuted = np.argmax(y_pred_permuted, axis=1)

    # シャッフル後のAccuracyを計算
    permuted_accuracy = accuracy_score(y_true_classes, y_pred_classes_permuted)

    # 性能の差を計算（重要度）
    importance = baseline_accuracy - permuted_accuracy
    feature_importances.append(importance)
    print(f'Feature {feature_idx + 1} Importance: {importance:.4f}')

# 8.3 特徴量重要度の可視化
feature_names = [f'Feature {i+1}' for i in range(num_features)]

plt.figure(figsize=(10, 6))
plt.barh(X[7].columns, feature_importances)
plt.xlabel('Decrease in Accuracy')
plt.title('Permutation Feature Importance')
plt.gca().invert_yaxis()
plt.show()


In [ ]:
#  pickleファイルを読み込む
with open('/Users/hinase/Downloads/Th-s/d_nakazawa_acc_check_segments4.7new.pkl', 'rb') as f:
    data = pickle.load(f)

segx = data['d_nakazawa_check_segx']
segy = data['d_nakazawa_check_segy']
segz = data['d_nakazawa_check_segz']

In [ ]:
# Create a simple sine wave as an example of a waveform or signal
x = np.linspace(0, 10, 1000)
y = np.sin(x)

# Define the sections to be colored
color_sections = [(200, 300, 'red'), (450, 550, 'blue'), (700, 800, 'red')]

# Plot the entire waveform in gray
plt.figure(figsize=(10, 2))
plt.plot(x, y, color='gray', linewidth=3)

# Highlight the specified sections with the chosen colors
for start, end, color in color_sections:
    plt.plot(x[start:end], y[start:end], color=color, linewidth=5)

# Add an arrow to indicate the flow of data
# plt.annotate('', xy=(10, 0), xytext=(0, 0),
#              arrowprops=dict(facecolor='black', shrink=0.05, width=2))
plt.xticks(color="None")
plt.yticks(color="None")
plt.tick_params(length=0)
# Remove axes for a cleaner look
plt.axis('off')
#plt.savefig("/Users/hinase/Downloads/plt.svg")

# Show plot
plt.show()

In [ ]:
import cairosvg

# SVGファイルのパス
input_svg_path = "/Users/hinase/Downloads/ss.svg"
# 出力するEPSファイルのパス
output_eps_path = "/Users/hinase/Downloads/ss.eps"

# SVGをEPSに変換
cairosvg.svg2eps(url=input_svg_path, write_to=output_eps_path)

print(f"SVG画像がEPS形式で '{output_eps_path}' に保存されました。")


In [6]:
import os
import csv
import openpyxl
from natsort import natsorted

folder_path = '/Users/hinase/Downloads/folder4'
folder_2_path = '/Users/hinase/Downloads/folder5'

# Get Excel file names in natural order, filtering only supported Excel files
supported_extensions = ('.xlsx', '.xlsm', '.xltx', '.xltm')
files = natsorted([f for f in os.listdir(folder_path) if f.endswith(supported_extensions)])

# Loop over filenames and build absolute path for each file
for filename in files:
    filepath = os.path.join(folder_path, filename)

    # Access xlsx file and get the first sheet object
    wb = openpyxl.load_workbook(filepath)
    ws_name = wb.sheetnames[0]
    ws = wb[ws_name]

    # Convert to csv and save to folder4
    savecsv_path = os.path.join(folder_2_path, filename.rstrip(".xlsx") + ".csv")
    with open(savecsv_path, 'w', newline="") as csvfile:
        writer = csv.writer(csvfile)
        for row in ws.rows:
            writer.writerow([cell.value for cell in row])


In [6]:
import numpy as np
import pylab as plt
import pandas as pd
import matplotlib.gridspec as gridspec
from tslearn.metrics import dtw_path

In [7]:
data = pd.read_csv("./data.csv", header=None)[1].values

In [ ]:
X = data[:1000:4]
Y = data[1000::4]

plt.figure(figsize=(6, 5))
plt.plot(X, label="Stream Data")
plt.plot(Y, label="Test Data")
plt.legend()
# PDFで保存
plt.savefig("/Users/hinase/Downloads/spring.pdf", format="pdf")  # PDF形式で保存
plt.show()  # グラフを表示

In [ ]:
from spring import spring
plt.figure(figsize=(6, 5))
# ループ内では凡例を設定せず、プロットのみ行う
for path, cost in spring(X, Y, 80):
    plt.plot(X, c="gray", alpha=0.5)  # 灰色の線
    plt.plot(path[:, 0], X[path[:, 0]], c="C2")  # 緑色の線

# ループの外で凡例を1回だけ設定
plt.plot([], [], c="C2", label="Segments identified as gesture performed")  # ダミーラインで凡例を作成
plt.legend(loc="upper right")  # 凡例を右上に固定
plt.savefig("/Users/hinase/Downloads/afterspring.pdf", format="pdf")  # PDF形式で保存
plt.show()  # グラフを表示